In [ ]:
#Importing Modules
import re, string, unicodedata
import nltk
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from gensim.summarization.summarizer import summarize    # 3.8.3

import glob
import textract
import warnings
import PyPDF2


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')


# Making a class for the final output object
class ResultElement:
    def __init__(self, rank, filename):
        self.rank = rank
        self.filename = filename
# Function to give file location
def getfilepath(loc):
    temp = str(loc)
    temp = temp.replace('\\', '/')
    return temp

# Function to remove non ascii words from text
def remove_non_ascii(words):
    '''to remove all non-ASCII char from tokenized words'''
    words_res = []
    for word in words:
        word_new = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        words_res.append(word_new)
    return words_res

# Function to convert text from uppercase to lowercase
def to_lowercase(words):
    '''converting to lowercase '''
    words_res = []
    for word in words:
        word_new = word.lower()
        words_res.append(word_new)
    return words_res

# Function to remove punctuations
def remove_punctuation(words):
    ''' removing punctuations'''
    words_res = []
    for word in words:
        word_new = re.sub(r'[^\w\s]', '', word)
        if word_new != '':
            words_res.append(word_new)
    return words_res

# Function to replace the number with their English spellings
def replace_numbers(words):
    '''replacing integer in list of tokenized words to text form'''
    p = inflect.engine()
    words_res = []
    for word in words:
        if word.isdigit():
            word_new = p.number_to_words(word)
            words_res.append(word_new)
        else:
            words_res.append(word)
    return words_res


# Function to remove stopwords from the texts
def remove_stopwords(words):
    '''removing stopwords from tokenized word list'''
    words_res = []
    for word in words:
        if word not in stopwords.words('english'):
            words_res.append(word)
    return words_res

# Function to stem tokenized words
def stem_words(words):
    '''stemming words in tokenized words list'''
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

# Function to lemmatize tokenized words
def lemmatize_verbs(words):
    '''lemmatizing verbs in tokenized words list'''
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas


# A Normalization function used to run all the above functions on the text at once
def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    words = stem_words(words)
    words = lemmatize_verbs(words)
    return words


# lists declaration
Resume_Vector = []
Ordered_list_Resume = []
Ordered_list_Resume_Score = []
LIST_OF_FILES = []


#Separate lists for differentiating between pdf , doc, and docx file
LIST_OF_FILES_PDF = []
LIST_OF_FILES_DOC = []
LIST_OF_FILES_DOCX = []

Resumes_File_Names = []
Resumes = []


Temp_pdf = ''

# changing directory to folder where our resumes are stored

os.chdir(r'C:\Users\VISHWAS\Desktop\Wabatech Project\Project 2\My Progress')

# here in single quotes put the address of the folder where you have resumes in pdf, doc or docx format


#searching for pdf, doc and docs and separating them
for file in glob.glob('**/*.pdf', recursive=True):
    LIST_OF_FILES_PDF.append(file)
for file in glob.glob('**/*.doc', recursive=True):
    LIST_OF_FILES_DOC.append(file)
for file in glob.glob('**/*.docx', recursive=True):
    LIST_OF_FILES_DOCX.append(file)


#making one list out of pdf, doc and docx
LIST_OF_FILES = LIST_OF_FILES_DOC + LIST_OF_FILES_DOCX + LIST_OF_FILES_PDF

#checking all Resumes files name
print("\n")
print("-------------This is LIST OF FILES--------------------")
print("\n")


for i,j in enumerate(LIST_OF_FILES):
    print(str(i)+"----"+str(j))	 #printing each resume name with serial no
    print('\n')



#for going through each resume
for no,i in enumerate(LIST_OF_FILES):
    Ordered_list_Resume.append(i)
    Temp = i.split(".")
#if the resume is in pdf format, extracting the text
    if Temp[1] == "pdf" or Temp[1] == "Pdf" or Temp[1] == "PDF":
        try:
            print('----------------------------------------------------------------------------------')
            print("\n\nThis is PDF\n\n"+str(no)+"    "+str(i))
            with open(i,'rb') as pdf_file:
                read_pdf = PyPDF2.PdfFileReader(pdf_file,strict=False)
                # page = read_pdf.getPage(0)
                # page_content = page.extractText()
                # Resumes.append(Temp_pdf)

                number_of_pages = read_pdf.getNumPages()
                for page_number in range(number_of_pages):

                    page = read_pdf.getPage(page_number)
                    page_content = page.extractText()
                    page_content = page_content.replace('\n', ' ')
                    # page_content.replace("\r", "")
                    Temp_pdf = str(Temp_pdf) + str(page_content)
                    # Temp_pdf.append(page_content)
                    # print(Temp_pdf)



                '''
                Temp_pdf = nltk.word_tokenize(Temp_pdf)
                Temp_pdf = normalize(Temp_pdf)
                Temp_pdf = ' '.join(map(str, Temp_pdf))
                '''

                #print(Temp_pdf)


                #print('----------------------------------------------------------------------------------')



                Resumes.extend([Temp_pdf])
                Temp_pdf = ''
                # f = open(str(i)+str("+") , 'w')
                # f.write(page_content)
                # f.close()


        except Exception as e: print(e)

#if the resume is in doc, extracting text from it
    if Temp[1] == "doc" or Temp[1] == "Doc" or Temp[1] == "DOC":
        print("This is DOC\n" , i)

        try:
            a = textract.process(i)
            a = a.replace(b'\n',  b' ')
            a = a.replace(b'\r',  b' ')
            a = a.replace(b'\t',  b' ')
            b = str(a)

            '''
            b = nltk.word_tokenize(b)
            b = normalize(b)
            b = ' '.join(map(str, b))
            '''
            c = [b]


            #print(c)
            #print("\n\n\n\n\n")


            Resumes.extend(c)
        except Exception as e: print(e)

  #if the resume is in docx, extracting text from it

    if Temp[1] == "docx" or Temp[1] == "Docx" or Temp[1] == "DOCX":
        print("This is DOCX\n" , i)
        try:
            a = textract.process(i)
            a = a.replace(b'\n',  b' ')
            a = a.replace(b'\r',  b' ')
            a = a.replace(b'\t',  b' ')

            b = str(a)

            '''
            b = nltk.word_tokenize(b)
            b = normalize(b)
            b = ' '.join(map(str, b))
            '''

            c = [b]


            #print(c)
            #print("\n\n\n\n\n")


            Resumes.extend(c)
        except Exception as e: print(e)


    if Temp[1] == "ex" or Temp[1] == "Exe" or Temp[1] == "EXE":
        print("This is EXE" , i)
        pass



print("Done Parsing.")

#just cheking the correct working directory , can remove
print(os.getcwd())

'''
print("this is resumes list ----------")

count=0
for i in Resumes:

    print("--------------------------------------")
    print(i)
    count+=1
    print('\n\n')
print(count)
'''


# now job description part

Job_Desc = 0
LIST_OF_TXT_FILES = []



f = open("oaktree.txt", 'r')

#here put the address of the job description file(in my case it was the same folder where my code was stored, that’s why I directly given the name)

text = f.read()

f.close()


#extracting text from JD and summarizing it
try:
    t = str(text)
    t = summarize(t, word_count=200)

    t = nltk.word_tokenize(t)
    t = normalize(t)
    t = ' '.join(map(str, t))

    text = [t]

except:
    text = 'None'


print(text)
print("\n\n\n\n\n")



#final model, applying KNN
vectorizer = TfidfVectorizer(stop_words='english')
# print(text)
vectorizer.fit(text)
vector = vectorizer.transform(text)

Job_Desc = vector.toarray()


print(text)
print("\n\n\n\n\n")

count=0
for i in Resumes:
    count+=1
    text = i
    t = str(text)
    try:
        t = summarize(t)

        t = nltk.word_tokenize(t)
        t = normalize(t)
        t = ' '.join(map(str, t))




        text = [t]
        vector = vectorizer.transform(text)

        aaa = vector.toarray()
        '''

        print("\n\n\n\n\n")
        print(aaa)

        print("----------------------------------------------------")
        print(aaa)
        print("----------------------------------------------------")
        print('\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n')
        '''

        Resume_Vector.append(vector.toarray())

    except Exception as e:
        print("resume no ---"+str(count))
        print(e)


for i in Resume_Vector:

    samples = i
    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(samples)
    NearestNeighbors(algorithm='auto', leaf_size=30)

    Ordered_list_Resume_Score.extend(neigh.kneighbors(Job_Desc)[0][0].tolist())


Z = [x for _,x in sorted(zip(Ordered_list_Resume_Score,Ordered_list_Resume))]

print(Ordered_list_Resume)
print(Ordered_list_Resume_Score)

final_res = []


#printing the result, using the already defined class for result
for n,i in enumerate(Z):
    # print("Rank\t" , n+1, ":\t" , i)
    # final_res.append(str("Rank\t" , n+1, ":\t" , i))
    name = getfilepath(i)
    #name = name.split('.')[0]
    rank = n+1
    res = ResultElement(rank, name)
    final_res.append(res)
    # res.printresult()
    print(f"Rank{res.rank+1} :\t {res.filename}")
